In [82]:
raw_path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\4_CRF')
filename = ['VRABES_CRF_form.xlsx', 'VRABES_Demographic.xlsx']
sheet = ['baseline','normal','anxiety','Demographic']
drop = ['no','code','name']
# header = 3, index_col = 'ID'


In [162]:
with open('vrabes_beh_dict.pkl','rb') as f:
    data = pickle.load(f)
raw_df = pd.DataFrame(data).transpose()

In [91]:
def crf_pre (data):
    data.drop(data.columns[data.columns.str.contains('Unnamed')],axis =1 , inplace = True)
    data.drop([col for col in data.columns if any([x for x in drop if x == col])], axis=1, inplace=True)
    data.dropna(subset = ['ID'], inplace = True)
    return data

In [168]:
def gen_crf (raw_path):
    demo = pd.read_excel(os.path.join(raw_path,'VRABES_Demographic.xlsx'), sheet_name= 'Demographic', header = 3, index_col= 'no')
    ses_01_all = pd.read_excel(os.path.join(raw_path,'VRABES_CRF_form.xlsx'), sheet_name= 'baseline', header = 3, index_col= 'no')
    ses_02_nor = pd.read_excel(os.path.join(raw_path,'VRABES_CRF_form.xlsx'), sheet_name= 'normal', header = 3, index_col= 'no')
    anx = pd.read_excel(os.path.join(raw_path,'VRABES_CRF_form.xlsx'), sheet_name= 'anxiety', header = 3, index_col= 'no')

    demo = crf_pre(demo)
    ses_01_all = crf_pre(ses_01_all)
    ses_02_nor = crf_pre(ses_02_nor)
    anx = crf_pre(anx)

    i_ses02 = anx.columns.get_loc('visit_2_d')
    i_ses03 = anx.columns.get_loc('visit_3_d')
    i_ses04 = anx.columns.get_loc('visit_4_d')

    ses_02_anx = pd.concat([anx.iloc[:,0],anx.iloc[:,1:i_ses03]], axis=1)
    ses_03_anx = pd.concat([anx.iloc[:,0],anx.iloc[:,i_ses03:i_ses04]], axis=1)
    ses_04_anx = pd.concat([anx.iloc[:,0],anx.iloc[:,i_ses04:]], axis=1)

    ses_01_all.rename(columns = lambda x:'ses_01_' + x, inplace=True)
    ses_02_nor.rename(columns = lambda x:'ses_02_' + x, inplace=True)
    ses_02_anx.rename(columns = lambda x:'ses_02_' + x, inplace=True)
    ses_03_anx.rename(columns = lambda x:'ses_03_' + x, inplace=True)
    ses_04_anx.rename(columns = lambda x:'ses_04_' + x, inplace=True)

    ses_01_all['ses_01_ID'] = ses_01_all.iloc[:,0].apply(lambda x:x+'_ses-01')
    ses_02_nor['ses_02_ID'] = ses_02_nor.iloc[:,0].apply(lambda x:x+'_ses-02')
    ses_02_anx['ses_02_ID'] = ses_02_anx.iloc[:,0].apply(lambda x:x+'_ses-02')
    ses_03_anx['ses_03_ID'] = ses_03_anx.iloc[:,0].apply(lambda x:x+'_ses-03')
    ses_04_anx['ses_04_ID'] = ses_04_anx.iloc[:,0].apply(lambda x:x+'_ses-04')

    ses_01_all.rename(columns = {'ses_01_ID':'ID'}, inplace=True)
    ses_02_nor.rename(columns = {'ses_02_ID':'ID'}, inplace=True)
    ses_02_anx.rename(columns = {'ses_02_ID':'ID'}, inplace=True)
    ses_03_anx.rename(columns = {'ses_03_ID':'ID'}, inplace=True)
    ses_04_anx.rename(columns = {'ses_04_ID':'ID'}, inplace=True)

    return demo, ses_01_all, ses_02_nor, ses_02_anx, ses_03_anx, ses_04_anx


In [ ]:
demo, ses_01_all, ses_02_nor, ses_02_anx, ses_03_anx, ses_04_anx = gen_crf(raw_path)

In [185]:
raw_df['ID'] = raw_df.index
test = pd.merge(raw_df,ses_01_all, how='left', on ='ID')
test2 = pd.merge(test,ses_02_nor, how='left', on ='ID')
test3 = pd.merge(test2, ses_02_anx,how='left', on ='ID')
test4 = pd.merge(test3, ses_03_anx,how='left', on ='ID')
test5 = pd.merge(test4, ses_02_anx,how='left', on ='ID')

In [49]:
def ses_per_sub (raw_df):        
    ses_dict = {}
    ses_dict2 = {}
    for i in raw_df.index:
        sub = i.split('_')[0]
        ses = i.split('_')[1]
        if sub in ses_dict.keys():
            ses_dict[sub] = ses_dict[sub]+1
        else: ses_dict[sub] = 1
    
    for a in raw_df.index:
        sub = a.split('_')[0]
        ses_dict2[a] = ses_dict[sub]    

    return ses_dict2

In [51]:
def time_ext (raw_df):
    cols = []    
    for col in raw_df.columns:
        if col.split('_')[-1] == 'Start Time':
            cols.append(col)
        else: pass
    return cols 

In [54]:
def as_ext(raw_df):
    cols = []
    for col in raw_df.columns:
        if "AnxScr" in col.split('_'):
            cols.append(col)
        else: pass
    return cols

In [57]:
def hrv_result(raw_df):
    cols = []
    for col in raw_df.columns:
        if "Result" in col.split('_'):
            cols.append(col)
        else: pass
    return cols